LLE:
$$\frac{\partial A}{\partial t} = -(\frac{\kappa}{2} + i(\omega_0 - \omega_p))A + i\frac{1}{2}D_2\frac{\partial^2 A}{\partial \phi^2} + ig|A|^2A + \sqrt{\frac{\kappa_{ext} P_A}{\hbar \omega_0}}$$

Normalized:
$$\frac{\partial a}{\partial \tau} = -(1 + i\zeta) a + i d_2 \frac{\partial^2 a}{\partial \phi^2} + i|a|^2 a + f$$
$$\tau = \kappa t/2,\ \zeta = 2(\omega_0 - \omega_p)/\kappa,\ d_2 = D_2/\kappa,\ a = \sqrt{2g/\kappa} A$$

Numerical:
$$a^\prime = \mathcal{F}^{-1}\{e^{-(1 + i\zeta + i d_2 \mu^2) \delta t} * \mathcal{F}\{e^{i|a|^2 \delta t} a\}\} + f \delta t$$

Counter-propagating:
$$\frac{\partial a}{\partial \tau} = -(1 + i\zeta) a + i d_2 \frac{\partial^2 a}{\partial \phi^2} + i|a|^2 a + i2\langle|b|^2\rangle a + f_A + iJb$$
$$\frac{\partial b}{\partial \tau} = -(1 + i\zeta) b + i d_2 \frac{\partial^2 b}{\partial \phi^2} + i|b|^2 b + i2\langle|a|^2\rangle b + f_B + iJa$$

Co-propagating:
$$\frac{\partial a}{\partial \tau} = -(1 + i\zeta) a + i d_2 \frac{\partial^2 a}{\partial \phi^2} + i(|a|^2 + B|b|^2) a + f_A + iJb$$
$$\frac{\partial b}{\partial \tau} = -(1 + i\zeta) b + i d_2 \frac{\partial^2 b}{\partial \phi^2} + i(|b|^2 + B|a|^2) b + f_B + iJa$$

Code below import modules.

In [ ]:
# Modules import
import numpy as np
import sys
import time
from numba import jit, objmode
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from IPython import display


Code below set parameters.

In [ ]:
# Parameters setting
""""""
# Physical parameters
f_A = 5 # normalized pump A
f_B = 0 # normalized pump B
d_2 = 0.04 # normalized dispersion, D_2/kappa, positive for anomalous dispersion, negative for normal dispersion
J_back_r = 0.0 # coupling factor between forward and backward modes
noise_level = 1e-6
""""""
""""""
# Simulation parameters
zeta_ini = -5 - 0.0001 # initial detuning
zeta_end = +35 + 0.0001 # final detuning
iter_number = 10**6 # number of iterations
mode_number = 2**8 # number of divided intracavity slices
delta_t = 1e-4
random_seed = np.random.randint(0, 2**31) # seed for random noise
""""""
""""""
# Simulation options
plot_flag = False # whether to plot runtime fields
noise_flag = True # whether to add noise
nonstop_flag = False # whether use the last result as the initial condition for the next simulation
""""""


Code below do preparations

In [ ]:
# Simulation preparation
cProfile_test = False
zeta_step = (zeta_end - zeta_ini) / (iter_number - 1)
plot_interval = 5000
record_interval = iter_number // 10000
power_interval = max(iter_number // 1000000, 1)

D_int = np.zeros(mode_number, dtype=np.complex128)
for i in range(mode_number):
    D_int[i] = (i - mode_number / 2) ** 2 * d_2
D_int = np.fft.ifftshift(D_int)

time_str = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
print(time_str)

rng = np.random.default_rng(random_seed)


Code below run the simulation.

In [ ]:
# Running simulation
@jit(nopython=True)
def noise(mode_number, rng, noise_level = noise_level):
    white_noise = rng.standard_normal(mode_number) + 1j * rng.standard_normal(mode_number)
    return white_noise * noise_level


@jit(nopython=True)
def cal_power(x):
    mode_number = len(x)
    return np.sum(np.abs(x)**2) / mode_number


@jit(nopython=True)
def split_step(A_0, zeta, f, D_int, delta_t, B, B_avg_pow, J_back_r=0, noise_flag=False, rng = rng):
    A_1 = np.exp(1j * (np.abs(A_0)**2 + 2 * B_avg_pow) * delta_t) * A_0
    A_1_freq = np.fft.fft(A_1)
    A_2_freq = np.exp(-(1 + 1j * zeta + 1j * D_int) * delta_t) * A_1_freq
    A_2 = np.fft.ifft(A_2_freq)
    A_3 = A_2 + f * delta_t
    A_4 = A_3 + 1j * J_back_r * delta_t * B # backscattering term from backwards mode
    if noise_flag:
        A_4 += noise(mode_number, rng) * delta_t
    return A_4


################
# Main loop
@jit(nopython=True)
def main_loop(iter_number, plot_interval, record_interval, zeta_ini, zeta_step, zetas, A, B, f_A, f_B, D_int, delta_t, J_back_r, noise_flag, rng, record_power_A, record_power_B, record_waveform_A, record_waveform_B, power_interval):
    zeta = zeta_ini - zeta_step
    for i in range(iter_number):
        zeta = zeta + zeta_step
        power_A = cal_power(A)
        power_B = cal_power(B)
        A_new = split_step(A, zeta, f_A, D_int, delta_t, B, power_B, J_back_r, noise_flag, rng)
        B_new = split_step(B, zeta, f_B, D_int, delta_t, A, power_A, J_back_r, noise_flag, rng)
        A, B = A_new, B_new

        if i % power_interval == 0:
            zeta_index = i // power_interval
            zetas[zeta_index] = zeta
            record_power_A[zeta_index] = power_A
            record_power_B[zeta_index] = power_B

        if i % record_interval == 0:
            record_waveform_A[i // record_interval] = A
            record_waveform_B[i // record_interval] = B

        if plot_flag == True and i % plot_interval == 0:
            with objmode():
                figure_plot(A, B, i, zeta, ax, ax_freq, line_A, line_B, line_A_freq, line_B_freq)
################


def figure_plot(A, B, i, zeta, ax, ax_freq, line_A, line_B, line_A_freq, line_B_freq):
    line_A.set_ydata(np.abs(A))
    line_B.set_ydata(np.abs(B))
    y_max = np.max([np.max(np.abs(A)), np.max(line_A.get_ydata())])
    ax.set_ylim(0, 1.2 * y_max)
    ax.title.set_text(f"zeta = {zeta:.2f}, proc = {i / iter_number * 100:.2f}%, f_A = {f_A}, J = {J_back_r}")

    A_freq = np.fft.fftshift(np.fft.fft(A))
    B_freq = np.fft.fftshift(np.fft.fft(B))
    line_A_freq.set_ydata(np.abs(A_freq))
    line_B_freq.set_ydata(np.abs(B_freq))
    y_freq_max = np.max(np.abs(A_freq))
    ax_freq.set_ylim(np.min(np.abs(A_freq)), 1.2 * y_freq_max)
    ax_freq.set_yscale('log')

    line_A_phase.set_ydata(np.angle(A))
    line_B_phase.set_ydata(np.angle(B))

    fig.canvas.draw()
    fig.canvas.flush_events()
    display.clear_output(wait=True)
    display.display(fig)


# Initialization
if nonstop_flag and "A_saved" in locals() and "B_saved" in locals():
    A = locals()["A_saved"]
    B = locals()["B_saved"]
    print("Start from last running state")
else:
    A = noise(mode_number, rng)
    B = noise(mode_number, rng)
    print("Start from random noise")

A_freq = np.fft.fftshift(np.fft.fft(A))
B_freq = np.fft.fftshift(np.fft.fft(B))

zetas = np.zeros(iter_number // power_interval)
record_power_A = np.zeros(iter_number // power_interval)
record_power_B = np.zeros(iter_number // power_interval)
record_waveform_A = np.zeros((iter_number // record_interval, mode_number), dtype=np.complex128)
record_waveform_B = np.zeros((iter_number // record_interval, mode_number), dtype=np.complex128)

xs_freq = np.arange(-mode_number / 2, mode_number / 2)
if plot_flag:
    fig, axs = plt.subplots(4)
    fig.set_size_inches(5, 7)
    ax, ax_freq, ax_phase_A, ax_phase_B = axs[0], axs[1], axs[2], axs[3]
    line_A, = ax.plot(np.abs(A), alpha = 0.7)
    line_B, = ax.plot(np.abs(B), alpha = 0.7)
    line_A_freq, = ax_freq.plot(xs_freq, np.abs(A_freq), alpha = 0.7)
    line_B_freq, = ax_freq.plot(xs_freq, np.abs(B_freq), alpha = 0.7)
    line_A_phase, = ax_phase_A.plot(np.angle(A), alpha = 0.7)
    line_B_phase, = ax_phase_B.plot(np.angle(B), alpha = 0.7)
    display.clear_output(wait=True)
    display.display(fig)
    plt.close()

start_time = time.time()
print("Start main loop")
main_loop(iter_number, plot_interval, record_interval, zeta_ini, zeta_step, zetas, A, B, f_A, f_B, D_int, delta_t, J_back_r, noise_flag, rng, record_power_A, record_power_B, record_waveform_A, record_waveform_B, power_interval)
print("End main loop")
end_time = time.time()
print(f"Time used: {end_time - start_time:.2f} s")

A_saved = record_waveform_A[-1]
B_saved = record_waveform_B[-1]


Code below plot the figures.

In [ ]:
# Results plot
time_str = sys.argv[1]

print("length of zetas:", len(zetas))
print("length of record_waveform:", len(record_waveform_A))


# Plot power
plt.figure()
plt.plot(zetas, record_power_A, label=f'Power A, f_A = {f_A}', alpha = 0.7)
plt.plot(zetas, record_power_B, label=f'Power B, f_B = {f_B}', alpha = 0.7)
plt.xlim(zeta_ini, zeta_end)
plt.title(f"Power, J = {J_back_r}, d_2 = {d_2}")
plt.xlabel("detuning")
plt.legend(loc = "lower left")

# Plot waveform heatmap
record_freq_A = np.fft.fftshift(np.fft.fft(record_waveform_A, axis=1), axes=1)
record_freq_B = np.fft.fftshift(np.fft.fft(record_waveform_B, axis=1), axes=1)

plt.figure()
plt.imshow(np.abs(record_waveform_A.T), aspect='auto', extent=[zeta_ini, zeta_end, mode_number, 0])
# plt.colorbar()
plt.title("Waveform_A")
plt.xlabel("detuning")

plt.figure()
plt.imshow(np.abs(record_waveform_B.T), aspect='auto', extent=[zeta_ini, zeta_end, mode_number, 0])
# plt.colorbar()
plt.title("Waveform_B")
plt.xlabel("detuning")

# plot the frequency in a heatmap
plt.figure()
plt.imshow(np.abs(record_freq_A.T), aspect='auto', extent=[zeta_ini, zeta_end, mode_number / 2, -mode_number / 2])
# plt.colorbar()
plt.title("Frequency_A")
plt.xlabel("detuning")

plt.figure()
plt.imshow(np.abs(record_freq_B.T), aspect='auto', extent=[zeta_ini, zeta_end, mode_number / 2, -mode_number / 2])
# plt.colorbar()
plt.title("Frequency_B")
plt.xlabel("detuning")


Code below is an anytime viewer

In [ ]:
# Anytime viewer
length = len(record_waveform_A)
length_zetas = len(zetas)


# 创建滑块和按钮
iteration_slider = widgets.IntSlider(value=0, min=0, max=length-1, step=1, description='Iteration', layout=widgets.Layout(width='60%'))
button_minus = widgets.Button(description='-')
button_plus = widgets.Button(description='+')

# 输出区域，用于显示更新后的图形
output = widgets.Output()

# 更新按钮的行为
def decrease(b):
    iteration_slider.value = max(0, iteration_slider.value - 1)

def increase(b):
    iteration_slider.value = min(length - 1, iteration_slider.value + 1)

button_minus.on_click(decrease)
button_plus.on_click(increase)

ax_wave_max = max(np.abs(record_waveform_A).max(), np.abs(record_waveform_B).max())
ax_freq_min = 0.3 * min(np.abs(record_freq_A[-1])[0], np.abs(record_freq_B[-1])[0])
ax_freq_max = max(np.abs(record_freq_A).max(), np.abs(record_freq_B).max())

# 更新绘图的函数
def update_plot(change):
    iter = iteration_slider.value
    detuning = zetas[iter * length_zetas // length]
    
    with output:
        output.clear_output(wait=True)
        fig = plt.figure(figsize=(8, 7))  
        gs = GridSpec(2, 2, height_ratios=[1, 0.7])

        ax_wave = fig.add_subplot(gs[0, 0])
        ax_wave.set_ylim(0, ax_wave_max)
        ax_freq = fig.add_subplot(gs[0, 1])
        ax_freq.set_yscale('log')
        ax_freq.set_ylim(ax_freq_min, ax_freq_max)     
        ax_power = fig.add_subplot(gs[1, :])

        ax_power.plot(zetas, record_power_A, label=f'f_A = {f_A}', alpha=0.7)
        ax_power.plot(zetas, record_power_B, label=f'f_B = {f_B}', alpha=0.7)
        ax_power.legend(loc="lower left", fontsize=8)
        ax_power.set_xlim(zeta_ini, zeta_end)
        ax_power.set_title(f"Power, J = {J_back_r}, d_2 = {d_2}")
        ax_power.axvline(x=detuning, color='red', linestyle='--', label=None)

        line_A, = ax_wave.plot(np.abs(record_waveform_A[iter]), alpha=0.7)
        line_B, = ax_wave.plot(np.abs(record_waveform_B[iter]), alpha=0.7)
        ax_wave.set_title(f"Iteration: {iter}, detuning: {detuning:.2f}")
        line_A_freq, = ax_freq.plot(xs_freq, np.abs(record_freq_A[iter]), alpha=0.7)
        line_B_freq, = ax_freq.plot(xs_freq, np.abs(record_freq_B[iter]), alpha=0.7)

        fig.tight_layout()
        plt.show()

# 绑定滑块的更新事件
iteration_slider.observe(update_plot, names='value')

# 显示滑块、按钮和输出区域
display.display(iteration_slider, widgets.HBox([button_minus, button_plus]), output)

# 初始绘制
update_plot(None)


Code below is temporary

In [ ]:
# Temporary codes
iter = 6466

dispersion_raw = d_2 * xs_freq**2
dispersion_A = d_2 * xs_freq**2 - (2 * cal_power(record_waveform_A[iter]) - np.abs(record_freq_A[iter])**2 / mode_number**2)
dispersion_B = d_2 * xs_freq**2 - (2 * cal_power(record_waveform_B[iter]) - np.abs(record_freq_B[iter])**2 / mode_number**2)

plt.figure()
plt.plot(xs_freq, dispersion_raw, label='raw')
plt.plot(xs_freq, dispersion_A, label=f'f_A = {f_A}')
plt.plot(xs_freq, dispersion_B, label=f'f_B = {f_B}')
plt.xlim(-mode_number / 20, mode_number / 20)
plt.ylim(-10, 10)
plt.xlabel("Frequency")
plt.ylabel("Dispersion")
plt.legend(loc="upper left")
plt.title(f"Dispersion, Iteration: {iter}, J = {J_back_r}")

round = 143-128
plt.figure()
temp = np.fft.fftshift(np.fft.fft(np.roll(record_waveform_A[iter], round)))
plt.plot(xs_freq, np.angle(temp))
plt.show()


Code block below calculate normalized parameters

In [ ]:
# Parameters calculation
def cal_parameters():
    c_0 = 3e8
    lambda_0 = 1550e-9
    omega_0 = 2 * np.pi * c_0 / lambda_0
    n_0 = 2.00
    n_2 = 22e-20
    Q_0 = 1e6
    FSR = 200e9
    thick = 0.8e-6
    width = 1.8e-6
    V_eff = c_0 / n_0 / FSR * width * thick
    P_in = 500e-3
    P_th = (n_0**2 * omega_0 * V_eff) / (n_2 * Q_0**2 * c_0)
    f = np.sqrt(P_in / P_th)
    print(f"f = {f}")

    return

cal_parameters()
